# Milestone 1 Notebook
- This is the same notebook we used for Milestone 1, except all class definitions have been moved to their own files and imported in. 
- This notebook is just to verify that our new code organisation doesn't break anything.
- Feel free to modify this notebook. Don't modify the MILESTONE_1_ARCHIVE.ipynb file.

In [1]:
import RPi.GPIO as GPIO
import gpiozero
import time
import numpy as np
import warnings
from matplotlib import pyplot as plt
from matplotlib.patches import Arrow
from IPython import display
import cv2
import threading
import random


In [ ]:
from robot_core.hardware.pin_config import *
from robot_core.hardware.diff_drive_robot import DiffDriveRobot
from robot_core.control.PI_controller import PIController
from robot_core.perception.opencv_detector import TennisBallDetector
from robot_core.motion.tentacle_planner import TentaclePlanner
from robot_core.orchestration.orchestrator import Orchestrator
from robot_core.utils.update_plot import update_plot


In [8]:
orchestrator = Orchestrator()


In [ ]:
orchestrator.start()


# Set up the plot
%matplotlib inline
plt.ion()  # Turn on interactive mode

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Main loop for updating the plot
try:
    while True:
        pass
#         update_plot(orchestrator, fig, axes)
#         time.sleep(3)  # Update plot every 3 seconds
except KeyboardInterrupt:
    print("Stopping the simulation...")
finally:
    orchestrator.stop()
    update_plot(orchestrator, fig, axes)
    plt.close(fig)


Initialising robot.
Initialising controller.
Initialising tennis ball detector.
Initialising tentacle planner.
Moving to next scanning position: col: 0, row: 0
Rotate method entered.

Beginning rotational scan now.
Ball detected. Distance from center: -102.00
Rotating by 0.17453292519943295 radians to adjust.

Rotate method entered.
Attempting to approach ball now. 
Moving forward 0.05 meters (ball_y = 50)
Ball not detected. Cannot approach.
Going home now.
Stopping the simulation...
Ended run. Robot ran for: 100.37 s


In [ ]:
# To regenerat e the plot in a nice way, run this cell:
%matplotlib inline
plt.ion()  # Turn on interactive mode

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
update_plot(orchestrator, fig, axes)

In [ ]:

robot = DiffDriveRobot(0.05, real_time=True)

In [ ]:
# Here are some example position commands we can put in.

#goal = (0.5, 0.5, 0)
#goal = (2, 2, np.pi/2)
goal = (1, 1, 0)
# goal = (0, 0, 60/180 * np.pi)




# Initialising controller, logging variables 
# Controller Parameters
controller = PIController(real_time=True) # using default Kp = 8, Ki=9
planner = TentaclePlanner() # remember to paste in the class in the cell where I told you


robot.x = 0
robot.y = 0
robot.th = 0
robot.wl = 0
robot.wr = 0

poses = []
goal_positions = []
velocities = []
desired_velocities = []
duty_cycle_commands = []
error_sums = []
errors = []
actual_dts = []

count = 0


try:
    goal_positions.append([0, 0, 0])
    goal_positions.append([goal[0],goal[1],goal[2]])

    start_time = time.time()
    while True: 
        
        ball_found = self.scan_for_ball(scan_direction=scan_direction, simulate_camera=False)
        if ball_found:
            self.approach_ball()
            self.navigate(0, 0, self.robot.th) # go home

        #debugging
        print(f"Pose: x={x:.2f}, y={y:.2f}, th={th:.2f}")
        print(f"Control Inputs: v={inputs['linear_velocity']:.2f}, w={inputs['angular_velocity']:.2f}")
        print(f"Duty Cycles: left={duty_cycle_l:.2f}, right={duty_cycle_r:.2f}")
        
        
        # Log data
        # Here, I record and log everything so we can graph it.
        poses.append([x,y,th])
        duty_cycle_commands.append([duty_cycle_l,duty_cycle_r])
        velocities.append([robot.wl,robot.wr])
        desired_velocities.append([wl_desired, wr_desired])
        count += 1
        time.sleep(robot.dt)
        print(count)
        
        break

except KeyboardInterrupt:
    print("Stopping robot")
    robot.set_motor_speed(0, 0)

    duration = time.time() - start_time

    # Plot robot data
#     display.clear_output(wait=True)
    fig = plt.figure(figsize=(15,9))
    
    plt.subplot(2, 2, 1)
    plt.plot(np.array(poses)[:,0],np.array(poses)[:,1], 'b', label='Robot Path')
    plt.plot(np.array(goal_positions)[:,0],np.array(goal_positions)[:,1],'ro', label='Goal Position')
    plt.plot(x,y,'k',marker='+')
    plt.quiver(x,y,0.1*np.cos(th),0.1*np.sin(th))
    plt.xlabel('x-position (m)')
    plt.xticks(np.arange(0, 5, 0.5))
    plt.yticks(np.arange(0, 5, 0.5))
    plt.ylabel('y-position (m)')
    plt.title(f"Ran for {duration:.2f} secs")
#     plt.axis('equal')
    plt.grid()

    plt.subplot(2, 2, 2)
    duty_cycle_commands = np.array(duty_cycle_commands)
    plt.plot(duty_cycle_commands[:, 0], label='Left Wheel')
    plt.plot(duty_cycle_commands[:, 1], label='Right Wheel')
    plt.xlabel('Time Step')
    
    plt.ylabel('Duty Cycle')
    plt.title('Duty Cycle Commands Over Time')
    plt.legend() 
    plt.grid()
    
    plt.subplot(2, 2, 3)
    velocities = np.array(velocities)
    desired_velocities = np.array(desired_velocities)
    plt.plot(velocities[:, 0], label='Left Wheel')
    plt.plot(velocities[:, 1], label='Right Wheel')
    plt.plot(desired_velocities[:, 0], label='Desired Left Wheel')
    plt.plot(desired_velocities[:, 1], label='Desired Right Wheel')
    plt.xlabel('Time Step')
    plt.ylabel('Wheel Velocity (rad/s)')
    plt.title('Wheel Velocity vs. Time')
    plt.legend() 
    plt.grid()
   
        
    fig.tight_layout()

    display.display(plt.gcf())

#     GPIO.cleanup()

    """
    Hopefully the graphs are self explanatory. If not, please ask me. I'm happy to explain anything.
    """
